# Steering along navigation route

In [45]:
print("Hello there")

Hello there


In [46]:
import carla
import time
import numpy as np
import math
import sys
import cv2

In [47]:
#connect to simulator
client = carla.Client('localhost', 2000)
world = client.get_world()

In [48]:
#utility script for destruction, clear all the actors (basically undo or destroy the vehicle)
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
#remove top and bottom qoutes to execute

In [49]:
'''
spectator = world.get_spectator()
transform = spectator.get_transform()
spectator.set_transform(carla.Transform())
'''

'\nspectator = world.get_spectator()\ntransform = spectator.get_transform()\nspectator.set_transform(carla.Transform())\n'

In [50]:
'''
#print all the vehicles
for bp in world.get_blueprint_library().filter('vehicle'):
    print(bp.id)

vehicle.tesla.model3
'''

"\n#print all the vehicles\nfor bp in world.get_blueprint_library().filter('vehicle'):\n    print(bp.id)\n\nvehicle.tesla.model3\n"

In [51]:
# get the spawn points of world
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[0]

In [52]:
#look for a blue print of your desired car, i want tesla
vehicle_bp = world.get_blueprint_library().filter('vehicle.tesla.model3')
#print(vehicle_bp)

#spawn the vehicle on start point
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

In [53]:
# position and create camera and open a live stream of car
CAMERA_POS_Z = 3
CAMERA_POS_X = -5

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x=CAMERA_POS_X))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to = vehicle)

def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {
    'image': np.zeros((image_h, image_w, 4))
}

camera.listen(lambda image: camera_callback(image, camera_data))

In [54]:
# define constants 

#define speed constants
PREFERRED_SPEED = 30
SPEED_TRESHOLD = 2

#adding parameters to display text to the image
font = cv2.FONT_HERSHEY_SIMPLEX

#defining the lines and font to display
org = (30, 30) # this line will be used to show current speed
org2 = (30, 50) # this line will be used for future steering angle
org3 = (30, 70) # and another line for future telemetry outputs
org4 = (30, 90) # and another line for future telemetry outputs
org3 = (30, 110) # and another line for future telemetry outputs
fontScale = 0.5
color = (255, 255, 255)
thickness = 1

In [55]:
# function to miantain the speed
def maintain_speed(s):
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_TRESHOLD:
        return 0.8
    else:
        return 0.4

In [57]:
#import route planner code from the simulation library path
import sys
sys.path.append('G:\CARLA_0.9.14\WindowsNoEditor\PythonAPI\carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner

point_a = start_point.location

#draw the longest possible route
sampling_resolution = 2

grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)

distance = 0
for loc in spawn_points: 
    cur_route = grp.trace_route(point_a, loc.location)
    if len(cur_route) > distance: 
        distance = len(cur_route)
        route = cur_route

#printing the route on map in the form of waypoints
for waypoint in route:
    world.debug.draw_string(waypoint[0].transform.location, 'O', draw_shadow=False, 
                            color = carla.Color(r=0, g=255, b=0), life_time = 120.0,
                            persistent_lines = True)

ModuleNotFoundError: No module named 'agents'

In [32]:
cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])

vehicle.set_autopilot(True)

#main loop 
quit = False

while True:
    # Carla Tick
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        vehicle.set_autopilot(False)
        vehicle.apply_control(carla.VehicleControl(throttle=0, 
                                                   steer=0,
                                                  brake=1))
        quit = True
        break
    image = camera_data['image']
    
    #get current speed
    v = vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    # now we add the speed to the window showing a camera mounted on the car
    image = cv2.putText(image, 'Speed: '+str(int(speed))+' kmh', org2, 
                        font, fontScale, color, thickness, cv2.LINE_AA)

    # show forward vector values (z dimension is ignored as we are not navigating up or down)
    fwd_vector = vehicle.get_transform().get_forward_vector()
    # now we overlay x and y
    image = cv2.putText(image, 'Fwd_vec.x: '+str(round(fwd_vector.x,3)), org3, 
                        font, fontScale, color, thickness, cv2.LINE_AA)
    image = cv2.putText(image, 'Fwd_vec.y: '+str(round(fwd_vector.y,3)), org4, 
                        font, fontScale, color, thickness, cv2.LINE_AA)

    cv2.imshow('RGB Camera',image)

#clean up
cv2.destroyAllWindows()
camera.stop()

for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [20]:
world.debug.draw_string(route[25][0].transform.location, '25', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=600.0,
        persistent_lines=True)
world.debug.draw_string(route[50][0].transform.location, '50', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=600.0,
        persistent_lines=True)
world.debug.draw_string(route[75][0].transform.location, '75', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=600.0,
        persistent_lines=True)
world.debug.draw_string(route[95][0].transform.location, '95', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=600.0,
        persistent_lines=True)
world.debug.draw_string(route[120][0].transform.location, '120', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=600.0,
        persistent_lines=True)
world.debug.draw_string(route[150][0].transform.location, '150', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=600.0,
        persistent_lines=True)

world.debug.draw_string(route[175][0].transform.location, '175', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=600.0,
        persistent_lines=True)
world.debug.draw_string(route[200][0].transform.location, '200', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=600.0,
        persistent_lines=True)
world.debug.draw_string(route[225][0].transform.location, '225', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=600.0,
        persistent_lines=True)

NameError: name 'route' is not defined

In [19]:
wp_vector = route[25][0].transform.get_forward_vector()

# get the cordinates of the car
car_x = vehicle.get_transform().location.x
car_y = vehicle.get_transform().location.y

wp_x = route[25][0].transform.location.x
wp_y = route[25][0].transform.location.y

print('waypoint 25 forward vector.x: ',wp_vector.x)
print('waypoint 25 forward vector.y: ',wp_vector.y)
print('waypoint 25 location x: ',wp_x)
print('waypoint 25 location y: ',wp_y)
print('car location x: ',car_x)
print('car location y: ',car_y)

NameError: name 'route' is not defined

In [31]:
# calculate the angle and illustrate it
angle = math.degrees(math.atan((wp_y - car_y)/(wp_x - car_x)))
print("angle to way point: ", angle)

angle to way point:  32.87534712262566
